In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [28]:
class GlaucomaIdentifier:
    def __init__(self, img, discDiameter=1, cupDiameter=0):
        self.image = img
        self.discDiameter = discDiameter
        self.cupDiameter = cupDiameter
        
    def getYComponent(self,pixel):
        #Converts RGB pixel to YCrcb and gets the Y component
        #@param pixel
        #@return Y component of the pixel
        blue, green, red = pixel
        #print("pixel"+str(pixel))
        yComponent = ( (red*0.299) + (green*0.587) + (blue*0.144) )
        #print(yComponent)
        return yComponent
    
    def getVCDR(self):
        #Calculates Vertical Cup to Disc ratio (VCDR)
        #@return VCDR
        vcdr = float(self.cupDiameter) / float(self.discDiameter)
        vcdr = np.around(vcdr * 100.00)/100.00
        return vcdr
    
    def getCupImage(self):
        #Segments cup region, changing all its pixels to black and calculates cup diameter
        #@return Image with cup region segmented
        cupImage = GlaucomaIdentifier(np.copy(self.image))
        minDisc = 99999
        maxDisc = -1
        height, width, _ = cupImage.image.shape

        for j in range(height):
            for i in range(width):
                if(cupImage.getYComponent(cupImage.image[j][i]) > 150.0):
                    print("yeah")
                    cupImage.image[j][i] = (0,0,0)
                    #print(cupImage.image[j][i])
        #cv2.imshow("cupImage",cupImage.image)           
        return cupImage.image


In [29]:
sample = cv2.imread("glaucoma.jpg")
resized_img = cv2.resize(sample,(480,360))
im = GlaucomaIdentifier(np.copy(resized_img))
#print(im.getYComponent(1,1,1))
#print(im.getVCDR())
height, width, _ = sample.shape
print(sample.shape)
print(im.image.shape)

cupImage = im.getCupImage()
pixel = sample[0][0]
print("sampleAntes"+str(resized_img[0][0]))
print("pixelAntes"+str(pixel))
print(pixel)
pixel = (0,0,0)
resized_img[0][0] = (0,0,0)
print(pixel)
print("sampleDepois"+str(resized_img[0][0]))
print("pixelDepois"+str(pixel))
cv2.imshow("original",resized_img)
cv2.imshow("cupImage",cupImage)
#print(cupImage == sample)
# waits for user to press any key 
# (this is necessary to avoid Python kernel form crashing) 
cv2.waitKey(0)
# closing all open windows 
cv2.destroyAllWindows()

(2136, 3216, 3)
(360, 480, 3)
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
